In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
attrition_df = pd.read_csv('HR-Employee-Attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [2]:
# Generate our categorical variable list
attrition_cat = attrition_df.dtypes[attrition_df.dtypes == "object"].index.tolist()

In [3]:
# Check the number of unique values in each column
attrition_df[attrition_cat].nunique()

Attrition         2
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
OverTime          2
dtype: int64

In [4]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(attrition_df[attrition_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(attrition_cat)
encode_df.head()

C:\Users\frien\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Attrition_No,Attrition_Yes,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [5]:
# Merge one-hot encoded features and drop the originals
attrition_df = attrition_df.merge(encode_df,left_index=True, right_index=True)
attrition_df = attrition_df.drop(attrition_cat,1)
attrition_df.head()

C:\Users\frien\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1102,1,2,1,1,2,94,3,2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,49,279,8,1,1,2,3,61,2,2,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,37,1373,2,2,1,4,4,92,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,33,1392,3,4,1,5,4,56,3,1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,27,591,2,1,1,7,1,40,3,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [6]:
# Split our preprocessed data into our features and target arrays
y = attrition_df["Attrition_Yes"].values
X = attrition_df.drop(["Attrition_Yes","Attrition_No"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

C:\Users\frien\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 448       
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [10]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
35/35 [==============================] - 1s 1ms/step - loss: 82.0432 - accuracy: 0.6597
Epoch 2/100
35/35 [==============================] - 0s 1ms/step - loss: 31.1037 - accuracy: 0.7269
Epoch 3/100
35/35 [==============================] - 0s 989us/step - loss: 14.2518 - accuracy: 0.7260
Epoch 4/100
35/35 [==============================] - 0s 1ms/step - loss: 10.7379 - accuracy: 0.7441
Epoch 5/100
35/35 [==============================] - 0s 1ms/step - loss: 7.8379 - accuracy: 0.7477
Epoch 6/100
35/35 [==============================] - 0s 1ms/step - loss: 6.5089 - accuracy: 0.7559
Epoch 7/100
35/35 [==============================] - 0s 1ms/step - loss: 6.2370 - accuracy: 0.7595
Epoch 8/100
35/35 [==============================] - 0s 1ms/step - loss: 5.1083 - accuracy: 0.7659
Epoch 9/100
35/35 [==============================] - 0s 981us/step - loss: 4.1846 - accuracy: 0.7486
Epoch 10/100
35/35 [==============================] - 0s 1ms/step - loss: 3.6430 - accuracy: 0.7713
E

35/35 [==============================] - 0s 1ms/step - loss: 0.9345 - accuracy: 0.7849
Epoch 83/100
35/35 [==============================] - 0s 1ms/step - loss: 0.7140 - accuracy: 0.7913
Epoch 84/100
35/35 [==============================] - 0s 1ms/step - loss: 0.6794 - accuracy: 0.8049
Epoch 85/100
35/35 [==============================] - 0s 1ms/step - loss: 0.7671 - accuracy: 0.8258
Epoch 86/100
35/35 [==============================] - 0s 990us/step - loss: 1.0119 - accuracy: 0.7722
Epoch 87/100
35/35 [==============================] - 0s 980us/step - loss: 0.6843 - accuracy: 0.8185
Epoch 88/100
35/35 [==============================] - 0s 1ms/step - loss: 0.8300 - accuracy: 0.7949
Epoch 89/100
35/35 [==============================] - 0s 866us/step - loss: 0.8764 - accuracy: 0.8058
Epoch 90/100
35/35 [==============================] - 0s 1ms/step - loss: 0.9947 - accuracy: 0.8040
Epoch 91/100
35/35 [==============================] - 0s 1ms/step - loss: 1.4810 - accuracy: 0.7704
Epoch 9

In [11]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.7614 - accuracy: 0.8777 - 116ms/epoch - 10ms/step
Loss: 0.7613502740859985, Accuracy: 0.8777173757553101


In [12]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
 1/35 [..............................] - ETA: 15s - loss: 0.6715 - accuracy: 0.7188
Epoch 1: saving model to checkpoints\weights.01.hdf5
35/35 [==============================] - 1s 2ms/step - loss: 0.5921 - accuracy: 0.7142
Epoch 2/100
 1/35 [..............................] - ETA: 0s - loss: 0.3999 - accuracy: 0.8438
Epoch 2: saving model to checkpoints\weights.02.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.5097 - accuracy: 0.7559
Epoch 3/100
 1/35 [..............................] - ETA: 0s - loss: 0.4182 - accuracy: 0.7812
Epoch 3: saving model to checkpoints\weights.03.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.4587 - accuracy: 0.8013
Epoch 4/100
 1/35 [..............................] - ETA: 0s - loss: 0.3525 - accuracy: 0.8125
Epoch 4: saving model to checkpoints\weights.04.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.4277 - accuracy: 0.8294
Epoch 5/100
 1/35 [..............................] - ETA: 0

Epoch 36/100
 1/35 [..............................] - ETA: 0s - loss: 0.3796 - accuracy: 0.8750
Epoch 36: saving model to checkpoints\weights.36.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.2543 - accuracy: 0.9111
Epoch 37/100
 1/35 [..............................] - ETA: 0s - loss: 0.1672 - accuracy: 0.9688
Epoch 37: saving model to checkpoints\weights.37.hdf5
35/35 [==============================] - 0s 2ms/step - loss: 0.2524 - accuracy: 0.9120
Epoch 38/100
 1/35 [..............................] - ETA: 0s - loss: 0.3487 - accuracy: 0.8438
Epoch 38: saving model to checkpoints\weights.38.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.2508 - accuracy: 0.9111
Epoch 39/100
 1/35 [..............................] - ETA: 0s - loss: 0.2889 - accuracy: 0.8438
Epoch 39: saving model to checkpoints\weights.39.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.2491 - accuracy: 0.9102
Epoch 40/100
 1/35 [..............................] 

35/35 [==============================] - 0s 1ms/step - loss: 0.2109 - accuracy: 0.9283
Epoch 71/100
 1/35 [..............................] - ETA: 0s - loss: 0.0736 - accuracy: 1.0000
Epoch 71: saving model to checkpoints\weights.71.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.2097 - accuracy: 0.9274
Epoch 72/100
 1/35 [..............................] - ETA: 0s - loss: 0.0973 - accuracy: 1.0000
Epoch 72: saving model to checkpoints\weights.72.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.2088 - accuracy: 0.9292
Epoch 73/100
 1/35 [..............................] - ETA: 0s - loss: 0.1724 - accuracy: 0.9688
Epoch 73: saving model to checkpoints\weights.73.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.2074 - accuracy: 0.9301
Epoch 74/100
 1/35 [..............................] - ETA: 0s - loss: 0.1590 - accuracy: 0.9688
Epoch 74: saving model to checkpoints\weights.74.hdf5
35/35 [==============================] - 0s 1ms/step

In [14]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.3242 - accuracy: 0.9049 - 132ms/epoch - 11ms/step
Loss: 0.3241746425628662, Accuracy: 0.904891312122345


In [15]:
# Export our model to HDF5 file
nn_new.save("trained_attrition.h5")

In [16]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('trained_attrition.h5')

In [17]:
# Evaluate the completed model using the test data
model_loss, model_accuracy = nn_imported.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.3242 - accuracy: 0.9049 - 92ms/epoch - 8ms/step
Loss: 0.3241746425628662, Accuracy: 0.904891312122345
